In [ ]:
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey
from sqlalchemy.orm import declarative_base, relationship, sessionmaker

#Criando a base ORM - object relational manager

Base = declarative_base()

In [2]:
class Departamento(Base):
    __tablename__ = "departamentos"

    id = Column(Integer, primary_key=True, autoincrement=True)
    nome = Column(String, nullable=False, unique=True)

    #Relacionamento com Funcionários (One-to-Many)
    funcionarios = relationship("Funcionario", back_populates="departamento")

class Funcionario(Base):
    __tablename__ = "funcionarios"

    id = Column(Integer, primary_key=True, autoincrement=True)
    nome = Column(String, nullable=False, unique=True)
    cargo = Column(String, nullable=False)
    salario = Column(Integer, nullable=False)
    departamento_id = Column(Integer, ForeignKey("departamentos.id"))

    #Relacionamento com Departamento (Many-to-One)
    departamento = relationship("Departamento", back_populates="funcionarios")

    #Relacionamento com Projetos (One to Many)
    projetos = relationship("Projeto", back_populates="funcionario")

class Projeto(Base):
    __tablename__ = "projetos"

    id = Column(Integer, primary_key=True, autoincrement=True)
    nome = Column(String, nullable=False, unique=True)
    descricao = Column(String)
    funcionario_id = Column(Integer, ForeignKey("funcionarios.id"))

    #Relacionamento com Funcionário (Many to One)
    funcionario = relationship("Funcionario", back_populates="projetos")


In [3]:
engine = create_engine("sqlite:///banco_aula09.db", echo=False)
#conectou ao banco de dados banco_aula09.db caso ele exista (caso contrário cria um novo)

Base.metadata.create_all(engine) #passando as 3 tabelas


In [ ]:
# Criar uma sessão

Session = sessionmaker(bind=engine) #classe session com as tabelas acima
session = Session() #criamos uma sessão, uma instância da classe sessão


In [ ]:
#Começando novamente!!!!

from sqlalchemy import create_engine, Column, Integer, String, ForeignKey
from sqlalchemy.orm import declarative_base, relationship, sessionmaker

# Criando a base ORM
Base = declarative_base()

class Departamento(Base):
    __tablename__ = "departamentos"

    id = Column(Integer, primary_key=True, autoincrement=True)
    nome = Column(String, nullable=False, unique=True)

    # Relacionamento com Funcionários (One-to-Many)
    funcionarios = relationship("Funcionario", back_populates="departamento")

class Funcionario(Base):
    __tablename__ = "funcionarios"

    id = Column(Integer, primary_key=True, autoincrement=True)
    nome = Column(String, nullable=False, unique=True)
    cargo = Column(String, nullable=False)
    salario = Column(Integer, nullable=False)
    departamento_id = Column(Integer, ForeignKey("departamentos.id"))

    # Relacionamento com Departamento (Many-to-One)
    departamento = relationship("Departamento", back_populates="funcionarios")

    # Relacionamento com Projetos (One to Many)
    projetos = relationship("Projeto", back_populates="funcionario")

class Projeto(Base):
    __tablename__ = "projetos"

    id = Column(Integer, primary_key=True, autoincrement=True)
    nome = Column(String, nullable=False, unique=True)
    descricao = Column(String)
    funcionario_id = Column(Integer, ForeignKey("funcionarios.id"))

    # Relacionamento com Funcionário (Many to One)
    funcionario = relationship("Funcionario", back_populates="projetos")

# Criando a conexão com o banco de dados
engine = create_engine("sqlite:///banco_aula09.db", echo=False)

# Criando as tabelas no banco de dados
Base.metadata.create_all(engine)

# Criando uma sessão
Session = sessionmaker(bind=engine)
session = Session()

# Inserindo dados na tabela "departamentos"
dep1 = Departamento(nome="TI")
dep2 = Departamento(nome="RH")
dep3 = Departamento(nome="Financeiro")

# Adicionando os departamentos na sessão e confirmando
session.add_all([dep1, dep2, dep3])
session.commit()  # Certifique-se de escrever corretamente!




In [2]:
# Crie e adicione funcionários ao banco de dados

funcionario1 = Funcionario(nome="Carlos", cargo="Desenvolvedor", salario=5000, departamento=dep1)
funcionario2 = Funcionario(nome="Ana", cargo="Analista de RH", salario=4000, departamento=dep2)
funcionario3 = Funcionario(nome="Mariana", cargo="Contadora", salario=4500, departamento=dep3)

session.add_all([funcionario1, funcionario2, funcionario3])
session.commit()

In [3]:
#Crie e adicione os projetos

proj1 = Projeto(nome="Sistema WEB", descricao="Desenvolvimento sistema WEB", funcionario=funcionario1)
proj2 = Projeto(nome="Organização Treinos", descricao="Desenvolvimento organização", funcionario=funcionario2)
proj3 = Projeto(nome="Planejamento", descricao="Planejamento sistema WEB", funcionario=funcionario2)
proj4 = Projeto(nome="Gestão Clientes", descricao="Controle clientes", funcionario=funcionario3)

session.add_all([proj1, proj2, proj3, proj4])
session.commit()

In [6]:
#Liste todos os funcionários cadastrados exibindo tbm seu cargo, departamento ao qual pertencem e os projetos associados

funcionarios = session.query(Funcionario).all() # me dê a tabela completa

#funcionarios - retorna uma classe

for func in funcionarios:
    print(func.nome, func.cargo, func.departamento.nome)
    for proj in func.projetos:
        print(proj.nome, proj.descricao)
    print()


Carlos Desenvolvedor TI
Sistema WEB Desenvolvimento sistema WEB

Ana Analista de RH RH
Organização Treinos Desenvolvimento organização
Planejamento Planejamento sistema WEB

Mariana Contadora Financeiro
Gestão Clientes Controle clientes



In [8]:
# ##########Aula de 19-03-25 / Continuação################
# Consultando Funcionários por Departamento - liste todos os departamentos cadastrados e, para cada um deles, exiba os funcionários associados, incluindo o cargo e o salário

departamento = session.query(Departamento).all() # retorna uma lista com todos os departamentos

for dep in departamento:
    print(f"Departamento: {dep.nome}")
    for func in dep.funcionarios:
        print(f"{func.nome}, cargo: {func.cargo}, salário: {func.salario}")
    print("-" * 40)



Departamento: TI
Carlos, cargo: Desenvolvedor, salário: 5000
----------------------------------------
Departamento: RH
Ana, cargo: Analista de RH, salário: 4000
----------------------------------------
Departamento: Financeiro
Mariana, cargo: Contadora, salário: 4500
----------------------------------------


In [10]:
# Atualizações e Remoções
# Atualize o salário do funcionário Carlos para 5500.

funcionario = session.query(Funcionario).filter_by(nome="Carlos").update({"salario": 5500})
session.commit()

In [ ]:
# Outra forma:

'''
funcionario1 = session.query(Funcionario).filter_by(nome="Carlos").first()
funcionario1.salario = 5500
session.commit()
'''

'Carlos'

In [14]:
# Remova a funcionária Mariana e verifique se o projeto vinculado a ela também foi removido

funcionario_remover = session.query(Funcionario).filter_by(nome="Mariana").first()
session.delete(funcionario_remover)
session.commit()


In [20]:
projetos = session.query(Projeto).all()

for proj in projetos:
    print(proj.nome, "-----> ",proj.descricao)
    if proj.funcionario is None:
        print("Projeto sem funcionário")
    else:
        print(proj.funcionario.nome)
    print()

Sistema WEB ----->  Desenvolvimento sistema WEB
Carlos

Organização Treinos ----->  Desenvolvimento organização
Ana

Planejamento ----->  Planejamento sistema WEB
Ana

Gestão Clientes ----->  Controle clientes
Projeto sem funcionário



In [ ]:
# Remova o projeto "Gestão Clientes"

projetos_remover = session.query(Projeto).filter_by(funcionario_id=funcionario_remover.id).all()
projetos_remover #retorna uma lista com todos os projetos da Mariana, está vazia pq Mariana foi removida

# removendo o projeto "Gestão Clientes"

for proj in projetos_remover:
    session.delete(proj)
session.commit()

# Verificando se o projeto foi removido

projetos = session.query(Projeto).all()

for proj in projetos:
    print(proj.nome, "-----> ",proj.descricao)
    if proj.funcionario is None:
        print("Projeto sem funcionário")
    else:
        print(proj.funcionario.nome)
    print()



Sistema WEB ----->  Desenvolvimento sistema WEB
Carlos

Organização Treinos ----->  Desenvolvimento organização
Ana

Planejamento ----->  Planejamento sistema WEB
Ana

Gestão Clientes ----->  Controle clientes
Projeto sem funcionário



In [25]:
# Transferir todos os funcionários do departamento de TI para o departamento de RH

dep_origem = session.query(Departamento).filter_by(nome="TI").first() #retorna o departamento de TI

funcionarios = session.query(Funcionario).filter_by(departamento_id=dep_origem.id).all() #retorna uma lista com todos os funcionários do departamento de TI

dep_destino = session.query(Departamento).filter_by(nome="RH").first() #retorna o departamento de RH

for func in funcionarios:
    func.departamento_id = dep_destino.id #altera o departamento de cada funcionário para o departamento de RH

session.commit()



In [26]:
#Confere se os funcionários foram transferidos

departamentos = session.query(Departamento).all()

for dep in departamentos:
    print(f"Departamento: {dep.nome}")
    for func in dep.funcionarios:
        print(f"{func.nome}, cargo: {func.cargo}, salário: {func.salario}")
    print("-" * 40)

Departamento: TI
----------------------------------------
Departamento: RH
Carlos, cargo: Desenvolvedor, salário: 5500
Ana, cargo: Analista de RH, salário: 4000
----------------------------------------
Departamento: Financeiro
----------------------------------------


In [27]:
# Exclusão e Reset do Banco
# exclua todas as tabelas do banco de dados sem remover o arquivo.

Base.metadata.drop_all(bind=engine)



In [33]:
# Exclua o banco de dados completamente, removendo o arquivo.

import os

session.close()  # fechando a sessão

# Fechar a conexão do engine
engine.dispose()

if os.path.exists("banco_aula09.db"):
    print("Arquivo banco_aula09.db removido com sucesso!")
    os.remove("banco_aula09.db")
else:
    print("Arquivo banco_aula09.db não encontrado!")


Arquivo banco_aula09.db removido com sucesso!
